# 07. MLOps: Monitoring Dashboard

This notebook creates a simple dashboard to visualize monitoring results.

## Setup

In [ ]:
import hopsworks
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')

# Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

## Read Performance Metrics

In [ ]:
# Get performance metrics feature group
performance_fg = fs.get_feature_group(name="model_performance_metrics", version=1)

# Read metrics
metrics = performance_fg.read()
print(f"Found {len(metrics)} performance records")
metrics.head()

## Create Simple Dashboard

In [ ]:
# Create dashboard with 2 panels
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Panel 1: RMSE over time
if len(metrics) > 1:
    ax1.plot(metrics['timestamp'], metrics['rmse'], marker='o')
    ax1.set_title('RMSE Over Time')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('RMSE')
    ax1.tick_params(axis='x', rotation=45)
else:
    # Show single point if only one record
    ax1.bar(['Latest'], [metrics['rmse'].iloc[0]])
    ax1.set_title('Latest RMSE')
    ax1.set_ylabel('RMSE')

# Panel 2: Prediction volume
ax2.bar(['Predictions'], [metrics['n_predictions'].sum()])
ax2.set_title('Total Predictions Logged')
ax2.set_ylabel('Count')

plt.tight_layout()
plt.show()

# Summary statistics
print("\nSummary:")
print(f"Average RMSE: {metrics['rmse'].mean():.2f}")
print(f"Total predictions monitored: {metrics['n_predictions'].sum()}")

## Check for Alerts

In [ ]:
# Simple alert logic
RMSE_THRESHOLD = 150  # Example threshold

latest_rmse = metrics['rmse'].iloc[-1]

print("Alert Status:")
if latest_rmse > RMSE_THRESHOLD:
    print(f"⚠️  ALERT: RMSE ({latest_rmse:.2f}) exceeds threshold ({RMSE_THRESHOLD})")
    print("   Consider retraining models")
else:
    print(f"✅ Performance OK: RMSE ({latest_rmse:.2f}) within threshold")

## Summary

✅ Created simple monitoring dashboard

What we showed:
- Performance trends over time
- Total prediction volume
- Simple alert logic

Next steps:
- Add more metrics (MAE, MAPE)
- Break down by item-location
- Connect to alerting system
- Schedule regular dashboard updates